# Experimento 2
***
- Conjunto de Dados: VinBigData
- Analisando o efeito do valor médio de duas redes treinadas.

### Importação dos pacotes

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
#from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import urllib.request as url

url.urlretrieve('https://raw.githubusercontent.com/Alyssonmach/cnn-lung-diseases/main/assets/lung_segmentation.py', 
                'lung_segmentation.py')
from lung_segmentation import segmentation

import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import cv2
import torch
import albumentations as A
from lungs_segmentation.pre_trained_models import create_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from lungs_segmentation.pre_trained_models import create_model
import lungs_segmentation.inference as inference
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import os

model_seg = create_model("resnet34", )

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model_seg = model_seg.to(device)

def segmentation(image, path = 'image.png'):

  image = tf.keras.preprocessing.image.array_to_img(image)
  tf.keras.preprocessing.image.save_img(path, image)
  image, mask = inference.inference(model_seg, path, 0.2)
  os.remove(path)
  for values_i in range(0, len(mask[0])):
    for values_j in range(0, len(mask[0])):
      if (mask[0, values_i, values_j] + mask[1, values_i, values_j]) == 0:
        image[values_i, values_j, 0] = 0
        image[values_i, values_j, 1] = 0
        image[values_i, values_j, 2] = 0
  
  return image

### Pré-processamento nos dados

In [3]:
# lendo os dados de um arquivo csv
dataframe = pd.read_csv('/content/drive/MyDrive/vinbigdata/train.csv')
# criando uma coluna com os caminhos relativos as imagens
dataframe['image_path'] = '/content/drive/MyDrive/vinbigdata/train/' + dataframe.image_id + '.jpg'

In [4]:
print('total de imagens disponíveis:', str(len(set(dataframe['image_path']))))

total de imagens disponíveis: 15000


In [5]:
# visualizando os casos disponíveis
dataframe['class_name'].value_counts()

No finding            31818
Aortic enlargement     7162
Cardiomegaly           5427
Pleural thickening     4842
Pulmonary fibrosis     4655
Nodule/Mass            2580
Lung Opacity           2483
Pleural effusion       2476
Other lesion           2203
Infiltration           1247
ILD                    1000
Calcification           960
Consolidation           556
Atelectasis             279
Pneumothorax            226
Name: class_name, dtype: int64

In [6]:
# removendo os casos não relativos a distúrbios pulmonares
dataframe = dataframe[dataframe.class_name != 'Aortic enlargement']
dataframe = dataframe[dataframe.class_name != 'Cardiomegaly']
dataframe = dataframe[dataframe.class_name != 'Other lesion']
dataframe = dataframe[dataframe.class_name != 'Consolidation']

In [7]:
# separando os casos rotulados como normais e anormais
normal_cases = dataframe[(dataframe.class_id == 14) & (dataframe.class_name == 'No finding')]
abnormal_cases = dataframe[(dataframe.class_id != 14) & (dataframe.class_name != 'No finding')]

print('total de dados após a filtração:', str(len(set(normal_cases['image_path'])) + len(set(abnormal_cases['image_path']))))

total de dados após a filtração: 13948


In [8]:
# removendo as imagens repetidas
normal_data = normal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )
abnormal_data = abnormal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )

# criando dataframes especifos com caminhos para as imagens e rótulos
normal_data['target'] = 'normal'
abnormal_data['target'] = 'abnormal'

In [9]:
print('quantidade de dados rotulados como normais:', len(normal_data))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 10606
quantidade de dados rotulados como anormais: 3342


In [10]:
# removendo 69% dos casos normais para balancear os dados
normal, _ = train_test_split(normal_data, test_size = 0.69, random_state = 42)

In [11]:
print('quantidade de dados rotulados como normais:', len(normal))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 3287
quantidade de dados rotulados como anormais: 3342


In [12]:
# concatenando os dataframes de casos normais e anormais
full_data = pd.concat([normal, abnormal_data])

In [13]:
# misturando todos os dados do dataframe e reiniciando os valores dos índices 
full_data = full_data.sample(frac = 1, axis = 0, random_state = 42).reset_index(drop=True)

In [14]:
# separando os dados de treinamento e de teste
train_df, test_df = train_test_split(full_data, stratify = full_data['target'],
                                     test_size = 0.2, random_state = 42)

In [15]:
# separando os dados de validação dos dados de treinamento
train_df, validation_df = train_test_split(train_df, stratify = train_df['target'],
                                           test_size = 0.2, random_state = 42)

In [16]:
# visualizando a quantidade de dados
print('quantidade de imagens de treinamento:', len(train_df['image_path']))
print('quantidade de rótulos de treinamento:', len(train_df['target']))
print('quantidade de imagens de teste:', len(test_df['image_path']))
print('quantidade de rótulos de teste:', len(test_df['target']))
print('quantidade de imagens de validação:', len(validation_df['image_path']))
print('quantidade de rótulos de validação:', len(validation_df['target']))

quantidade de imagens de treinamento: 4242
quantidade de rótulos de treinamento: 4242
quantidade de imagens de teste: 1326
quantidade de rótulos de teste: 1326
quantidade de imagens de validação: 1061
quantidade de rótulos de validação: 1061


In [17]:
# visualizando a quantidade de exemplos disponíveis nos dados de treinamento
train_df['class_name'].value_counts()

No finding            2104
Pleural thickening     586
Pulmonary fibrosis     473
Lung Opacity           297
Pleural effusion       223
Nodule/Mass            216
Infiltration           108
Calcification          104
ILD                     90
Atelectasis             24
Pneumothorax            17
Name: class_name, dtype: int64

In [18]:
# visualizando a quantidade de exemplos disponíveis nos dados de validação
validation_df['class_name'].value_counts()

No finding            526
Pleural thickening    143
Pulmonary fibrosis    125
Lung Opacity           71
Nodule/Mass            65
Pleural effusion       44
ILD                    25
Infiltration           23
Calcification          23
Atelectasis            11
Pneumothorax            5
Name: class_name, dtype: int64

In [19]:
# visualizando a quantidade de exemplos disponíveis nos dados de teste
test_df['class_name'].value_counts()

No finding            657
Pleural thickening    179
Pulmonary fibrosis    148
Lung Opacity           80
Nodule/Mass            66
Pleural effusion       65
Calcification          40
Infiltration           38
ILD                    37
Atelectasis            10
Pneumothorax            6
Name: class_name, dtype: int64

In [20]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(preprocessing_function = segmentation)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      directory = '',
                                                      x_col = 'image_path',
                                                      y_col = 'target',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (512, 512))
# criando o gerador de imagens de validação 
valid_generator = image_generator.flow_from_dataframe(
                                                      dataframe = validation_df,
                                                      directory = '.', 
                                                      x_col = 'image_path',
                                                      y_col = 'target',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (512, 512))

# normalizando as imagens de teste 
test_datagen = ImageDataGenerator(rescale = 1./255.)

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  directory = '.',
                                                  x_col = 'image_path',
                                                  y_col = 'target',
                                                  batch_size = 32,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (512, 512))

Found 4242 validated image filenames belonging to 2 classes.
Found 1061 validated image filenames belonging to 2 classes.
Found 1326 validated image filenames belonging to 2 classes.


### Preparando a rede neural convolucional

In [52]:
model = MobileNetV2(input_shape = (512, 512, 3), include_top = False, weights = 'imagenet', 
                    classes = 2, classifier_activation = 'softmax')
x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
x = tf.keras.layers.Dense(units = 512, activation = 'relu')(x)
x = tf.keras.layers.Dense(units = 216, activation = 'relu')(x)
x = tf.keras.layers.Dropout(rate = 0.2)(x)
x = tf.keras.layers.Dense(units = 2, activation = 'softmax')(x)

model = tf.keras.models.Model(inputs = model.input, outputs = x)

In [54]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "transferlearning_weights.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [55]:
# definindo um array de callbacks
callbacks = [checkpoint]

In [41]:
for indices in range(0,10):
  (x1, y1) = train_generator[indices]
  if indices == 0:
    x, y = x1, y1
  x, y = np.concatenate([x, x1]), np.concatenate([y, y1])

In [42]:
for indices in range(0,5):
  (z1, w1) = valid_generator[indices]
  if indices == 0:
    z, w = z1, w1
  z, w = np.concatenate([z, z1]), np.concatenate([w, w1])

In [43]:
for indices in range(0,5):
  (a1, b1) = test_generator[indices]
  if indices == 0:
    a, b = a1, b1
  a, b = np.concatenate([a, a1]), np.concatenate([b, b1])

In [56]:
# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001), loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [ ]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 4242 // 32, 
                              validation_data = valid_generator, validation_steps = 1061 // 32,
                              callbacks = callbacks, epochs = 10)

Epoch 1/10
 18/132 [===>..........................] - ETA: 1:32:17 - loss: 0.6371 - acc: 0.6454

### Preparando a rede neural convolucional

In [ ]:
# baixando os pesos treinados da rede inception
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
# referenciando o local em que os pesos estão armazenados
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# carregando a arquitetura inception pré-treinada
pre_trained_model = InceptionV3(input_shape = (512, 512, 3), 
                                include_top = False, 
                                weights = None)

# carregando os pesos treinados com outros dados 
pre_trained_model.load_weights(local_weights_file)

# definindo as flags iniciais  
pre_trained_model.trainable = True
set_trainable = False

# para a arquitetura inception, a rede será retreinada a partir da camada 'mixed8'
for layer in pre_trained_model.layers:
    if layer.name == 'mixed8':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# obtendo a última camada como sendo a nomeada por 'mixed7'
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [ ]:
# definindo uma camada de achatamento
x = layers.Flatten()(last_output)
# conecatando a rede uma camada com 1024 neurônios e função de ativação relu
x = layers.Dense(units = 1024, activation = 'relu')(x)     
# conecatando a rede uma camada com 128 neurônios e função de ativação relu
x = layers.Dense(units = 512, activation = 'relu')(x) 
# aplicando uma camada de dropout com uma taxa de 20% (normalização)
x = layers.Dropout(rate = 0.2)(x)                  
# adicionando uma camada de saída com um neurônio e uma função de ativação sigmoide
x = layers.Dense  (units = 2, activation = 'softmax')(x)           

# conecatando as camadas definidas acima com a arquitetura inception
model = Model(pre_trained_model.input, x) 

# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001), loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [ ]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "transferlearning_weights.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [ ]:
# definindo um array de callbacks
callbacks = [checkpoint]

In [ ]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 4242 // 32, 
                              validation_data = valid_generator, validation_steps = 1061 // 32,
                              callbacks = callbacks, epochs = 10)